## Datos individuales a estadisticas agregadas

In [9]:
import pandas as pd
import numpy as np
import json
import os

In [10]:
## En esta notebook estan ...

# El universo (base) de analisis pueden ser:
#     - [P] Personas
#     - [H] Hogares
#     - [M] Mayores de 24
#     - [Hp] Hogares en pobreza
#     - [Hi] Hogares en indigencia

# Ademas restringido a un espacio geografico:
#     - ['AMBA'] i.e. data.loc[data.AGLOMERADO.isin([33, 32, 2])]

# Las variables agrupadoras pueden ser: 
#     - ['Q', 'Grupo Etario']
#     - ['Q','Total pais']
#     - ['Q','AGLOMERADO']
#     - ['Q','PROV']
#     - ['Q','AGLO_si']
#     - ['Q','P0910']
#     - ['Q','DPTO']
#     - ['Q','Region']


# Las variables de resultado pueden ser: (variable, sintetico, base)
#     - ["Pobreza", "mean", "Personas"]
#     - ["Pobreza", "sum", "Personas"]
#     - ["Indigencia", "mean", "Personas"]
#     - ["Indigencia", "sum", "Personas"]


In [11]:
import datetime as dt

def sintetizar(data, grouper, base='Personas', frac=0.05):
    df = data.copy()
    df['Total'] = True
    df['timestamp'] = dt.datetime.today()
    df['AGLO_si'] = df.AGLOMERADO != 0
    df['Total_pais'] = True

    # Columnas comunes
    columns_to_groupby = ['Total', 'Pobreza', 'Indigencia']
    
    # Columnas específicas según base
    if 'P47T_persona' in df.columns:
        columns_to_groupby.append('P47T_persona')
    if 'P47T_hogar' in df.columns:
        columns_to_groupby.append('P47T_hogar')
    if 'CB_EQUIV' in df.columns:
        columns_to_groupby.extend(['CB_EQUIV', 'CBA', 'gap_indigencia', 'CBT', 'gap_pobreza'])

    agg_dict = {
        'Total': ['mean', 'sum'],
        'Pobreza': ['mean', 'sum'],
        'Indigencia': ['mean', 'sum']
    }
    
    if 'P47T_persona' in df.columns:
        agg_dict['P47T_persona'] = ['mean', q10, q25, 'median', q75, q90]
    if 'P47T_hogar' in df.columns:
        agg_dict['P47T_hogar'] = ['mean', q10, q25, 'median', q75, q90]
    if 'CB_EQUIV' in df.columns:
        agg_dict['CB_EQUIV'] = ['mean', 'median']
        agg_dict['CBA'] = ['mean', 'median']
        agg_dict['gap_indigencia'] = ['mean', 'median']
        agg_dict['CBT'] = ['mean', 'median']
        agg_dict['gap_pobreza'] = ['mean', 'median']

    df = df.groupby(grouper + ['timestamp'])[columns_to_groupby].agg(agg_dict)
    
    for col in ['Total', 'Pobreza', 'Indigencia']:
        df[(col, 'sum')] = (df[(col, 'sum')]/frac).round(1)
        df[(col, 'mean')] = df[(col, 'mean')].round(4)

    if 'P47T_persona' in df.columns:
        df['P47T_persona'] = df['P47T_persona'].round(-1).astype(int)
    if 'P47T_hogar' in df.columns:
        df['P47T_hogar'] = df['P47T_hogar'].round(-1).astype(int)

    agg_result = df.T.set_index(np.repeat(base, df.shape[1]), append=True)
    stacker_ix = [-i for i in range(len(grouper) + 1)]
    agg_result = agg_result.stack(level=stacker_ix).reset_index()
    
    agg_result = agg_result.rename(columns = {'level_0': 'observable', 'level_1': 'sintetico', 'level_2': 'base', 0: 'valor'})
    agg_result['frac'] = frac
    return agg_result

# Funciones de percentil
def q10(x):
    return x.quantile(0.1)

def q25(x):
    return x.quantile(0.25)

def q75(x):
    return x.quantile(0.75)

def q90(x):
    return x.quantile(0.9)


In [12]:
import pandas as pd

# Parámetros
frac = 0.05
Q = '2023-02-15'
experiment_tag = 'ARG'  # Asumiendo que esta es la etiqueta, según tu ejemplo

# Nombres de los archivos
personas_ingresos_Q_file = f'./../data/Pobreza/personas_ingresos_f{frac}_{Q}_{experiment_tag}.csv'
pobreza_hogares_file = f'./../data/Pobreza/pobreza_hogares_f{frac}_q{Q}.csv'
hogares_geo_file = f'./../data/Pobreza/hogares_geo_f{frac}_{Q.split("-")[0]}_{experiment_tag}.csv'

# Cargar los archivos (solo las primeras 5 filas)
personas_ingresos_Q = pd.read_csv(personas_ingresos_Q_file, nrows=5)
pobreza_hogares = pd.read_csv(pobreza_hogares_file, nrows=5)
hogares_geo = pd.read_csv(hogares_geo_file, nrows=5)

# Merges
info_personas = personas_ingresos_Q.merge(pobreza_hogares, on=['HOGAR_REF_ID', 'Q'], how='left').merge(hogares_geo, on='HOGAR_REF_ID', how='left')
info_hogares = pobreza_hogares.merge(hogares_geo, on='HOGAR_REF_ID', how='left')

# Mostrar columnas
print("Columnas de info_personas:", info_personas.columns)
print("\nColumnas de info_hogares:", info_hogares.columns)


Columnas de info_personas: Index(['ID', 'HOGAR_REF_ID', 'Q', 'P02', 'P03', 'P0910', 'P47T_persona',
       'P47T_hogar', 'CBA', 'CBT', 'CB_EQUIV', 'Pobreza', 'Indigencia',
       'gap_pobreza', 'gap_indigencia', 'RADIO_REF_ID', 'AGLOMERADO',
       'FRAC_REF_ID', 'DPTO', 'NOMDPTO', 'PROV', 'NOMPROV', 'Region',
       'COD_2010', 'distrito_id', 'seccion_id', 'seccion_nombre', 'circuito',
       'IN1', 'NAM'],
      dtype='object')

Columnas de info_hogares: Index(['HOGAR_REF_ID', 'Q', 'P47T_hogar', 'CBA', 'CBT', 'CB_EQUIV', 'Pobreza',
       'Indigencia', 'gap_pobreza', 'gap_indigencia', 'RADIO_REF_ID',
       'AGLOMERADO', 'FRAC_REF_ID', 'DPTO', 'NOMDPTO', 'PROV', 'NOMPROV',
       'Region', 'COD_2010', 'distrito_id', 'seccion_id', 'seccion_nombre',
       'circuito', 'IN1', 'NAM'],
      dtype='object')


### Carga de datos

In [13]:
import pandas as pd
import os

# Parámetros
frac = 0.05
experiment_tag = 'ARG'

# Ruta de los datos
path = './../data/Pobreza/'

In [14]:
# Comprobar y crear el directorio de resultados si no existe
results_path = './../data/results'
if not os.path.exists(results_path):
    os.makedirs(results_path)

def add_to_json(synth_func, data, grouper, base_str, frac):
    """Función para agregar datos a un archivo JSON."""
    filename = f'{results_path}/result_{base_str}_{"-".join(grouper)}_{frac}.json'
    
    df = synth_func(data, grouper, frac)
    
    # Si el archivo no existe, lo crea. Si existe, concatena los nuevos datos.
    if not os.path.exists(filename):
        df.to_json(filename, orient="records")
    else:
        df_ = pd.concat([df, pd.read_json(filename)])
        df_.to_json(filename, orient="records")

In [15]:

# Lista de agrupaciones para las transformaciones
# groupers = [['Q', 'AGLO_si']]
groupers = [['Q','Total_pais'], ['Q','AGLO_si'], ['Q','AGLOMERADO'], ['Q','Region'], ['Q','Region', 'AGLO_si'], 
            ['Q', 'Region', 'AGLOMERADO'], ['Q','PROV'], ['Q','DPTO'], ['Q', 'PROV', 'AGLO_si'], ['Q','P0910']]

# Lista de fechas Q que se van a procesar
Qs = ['2022-05-15', '2022-08-15', '2022-11-15', '2023-02-15']  # Aquí puedes agregar más fechas si es necesario

# Bucle para procesar cada valor de Q
for Q in Qs:
    print(f"Procesando fecha Q: {Q}")
    
    # Nombres de los archivos dependientes de Q
    personas_ingresos_Q_file = f'{path}personas_ingresos_f{frac}_{Q}_{experiment_tag}.csv'
    pobreza_hogares_file = f'{path}pobreza_hogares_f{frac}_q{Q}.csv'
    hogares_geo_file = f'{path}hogares_geo_f{frac}_{Q.split("-")[0]}_{experiment_tag}.csv'

    # Cargar los archivos
    personas_ingresos_Q = pd.read_csv(personas_ingresos_Q_file)
    pobreza_hogares = pd.read_csv(pobreza_hogares_file)
    hogares_geo = pd.read_csv(hogares_geo_file)

    # Fusiones para obtener los datasets consolidados
    info_personas = personas_ingresos_Q.merge(pobreza_hogares, on=['HOGAR_REF_ID', 'Q'], how='left').merge(hogares_geo, on='HOGAR_REF_ID', how='left')
    info_hogares = pobreza_hogares.merge(hogares_geo, on='HOGAR_REF_ID', how='left')

    # Agregar AGLO SI, y Total Pais.
    info_personas['AGLO_si'] = info_personas.AGLOMERADO != 0; info_personas['Total_pais'] = True
    info_hogares['AGLO_si'] = info_hogares.AGLOMERADO != 0; info_hogares['Total_pais'] = True

    # Aplicar transformaciones y guardar resultados
    for grouper in groupers:
        # Sintetizar para personas
        add_to_json(sintetizar, info_personas, grouper, base_str='P', frac=frac)
        add_to_json(sintetizar, info_personas[info_personas.AGLO_si], grouper, base_str='PAGLO', frac=frac)
        add_to_json(sintetizar, info_personas[info_personas.P03 >= 24], grouper, base_str='M24', frac=frac)
        
        # Sintetizar para hogares
        add_to_json(sintetizar, info_hogares, grouper, base_str='H', frac=frac)
        add_to_json(sintetizar, info_hogares[info_hogares.Pobreza], grouper, base_str='Hp', frac=frac)
        add_to_json(sintetizar, info_hogares[info_hogares.Indigencia], grouper, base_str='Hi', frac=frac)

Procesando fecha Q: 2022-05-15


/tmp/ipykernel_1411810/2818111489.py:21: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  hogares_geo = pd.read_csv(hogares_geo_file)


KeyError: 'P0910'

In [ ]:
# frac
xx

NameError: name 'xx' is not defined

In [ ]:
base_str = 'H'
for grouper in groupers:
    file = './../data/results/result_'+base_str+'_'+'-'.join(grouper)+'.json'
    print(file)
    info = pd.read_json(file).drop('timestamp', axis = 1).drop_duplicates()

    info['Q'] = pd.to_datetime(info['Q'])

    # Mean pobreza, indigencia
    info_ = info.loc[info.sintetico == 'mean'].set_index(['observable'] + grouper)['valor']
#     plot_data = info_.unstack([1]).T

    ## Columnas que no son ni el trimestre ni el valor. Identifican las series de tiempo.
    time_series_cols = list(col for col in info.columns if col not in ['Q', 'valor'])
    plot_data = info.groupby([pd.Grouper(freq = '6M', key='Q')] + time_series_cols).agg({'valor': 'mean'}).unstack(0)['valor'].T
    
    if len(plot_data) < 50:
        fig, axs = plt.subplots(1, figsize = (6, 3.5))
#         plot_data.plot(ax = axs, marker = '.')
        plot_data['Pobreza']['mean'].plot(ax = axs, marker = '.')
        axs.legend(loc = (1, 0))
        axs.set_ylim(0, 1)
        plt.show()

In [ ]:
for grouper in groupers:
    info = pd.read_json('./../data/results/result_'+base_str+'_'+'-'.join(grouper)+'.json').drop('timestamp', axis = 1).drop_duplicates()
    
    info['Q'] = pd.to_datetime(info['Q'])
    
    # Mean pobreza, indigencia
    
    info_ = info.loc[info.sintetico == 'mean'].set_index(['observable'] + grouper)['valor']
#     plot_data = info_.unstack([1]).T
    
    ## Columnas que no son ni el trimestre ni el valor. Identifican las series de tiempo.
    time_series_cols = list(col for col in info.columns if col not in ['Q', 'valor'])
    plot_data = info.groupby([pd.Grouper(freq = '6M', key='Q')] + time_series_cols).agg({'valor': 'mean'}).unstack(0)['valor'].T
    
    if len(plot_data) < 50:
        fig, axs = plt.subplots(1, figsize = (6, 3.5))
#         plot_data.plot(ax = axs, marker = '.')
        plot_data['Pobreza']['mean'].plot(ax = axs, marker = '.')
        axs.legend(loc = (1, 0))
        axs.set_ylim(0, 1)
        plt.show()
        
#     # Median salario

#     info_ = info.loc[info.sintetico == 'median'].set_index(['observable'] + grouper)['valor']

#     plot_data = info_.unstack([1]).T
    
#     if info_.unstack([1]).T.shape[1] < 50:
#         fig, axs = plt.subplots(1, figsize = (6, 3.5))
#         plot_data.plot(ax = axs, legend = False, marker = '.')
#         axs.legend(loc = (1, 0))
# #         axs.set_ylim(0, 1)
#         axs.set_ylim(0, 8000)
#         plt.show()

In [ ]:
### Se carga la informacion de condiciones de pobreza de cada cuatrimestre y computa el valor sintetico (ej. pobreza por provincia/aglomerado)

In [ ]:
## Se ahorra algo de tiempo eligiendo las columnas a importar..

# %%timeit
# pd.read_csv(f)

# 2.09 s ± 49.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

# %%timeit
# pd.read_csv(f, usecols = grouper + ['Pobreza', 'Indigencia', 'P03', 'P47T_persona', 'HOGAR_REF_ID', 'P47T_hogar', 'CBA', 'CBT', 'gap'])

# 1.31 s ± 11.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

#  pero parece que conviene usar el siguiente loop donde cada info trimestral que se carga se agrupa, y luego se juntan los resultados: